In [ ]:
import pandas as pd
import wrds
import numpy as np
from datetime import datetime
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
# Connect to WRDS
db = wrds.Connection()

In [6]:
# Setup directory structure
base_dir = "/Users/dyuan868/Desktop/gpas/4993/stat-4993-sp25/Data"
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Create main directories
dirs = [
    "1_raw_data", 
    "2_processed_data", 
    "3_financial_ratios", 
    "4_industry_datasets",
    "5_analysis"
]
for dir_name in dirs:
    os.makedirs(f"{base_dir}/{dir_name}", exist_ok=True)
print(f"Created directory structure in {base_dir}")

Created directory structure in /Users/dyuan868/Desktop/gpas/4993/stat-4993-sp25/Data


In [7]:
# 1. Fetch CRSP monthly stock data with ticker and industry information
print("Fetching CRSP monthly stock data...")
crsp_query = """
    SELECT a.permno,
           a.date,
           b.ticker,
           b.comnam AS company_name,
           b.siccd AS sic_code,      -- Standard Industrial Classification code
           a.ret AS monthly_return,
           a.prc AS price,
           ABS(a.prc) AS adj_price,
           a.vol AS volume,
           a.shrout AS shares_outstanding,  -- Add shares outstanding for market cap calculation
           b.shrcd AS share_code,
           b.exchcd AS exchange_code,
           CASE 
               WHEN b.exchcd = 1 THEN 'NYSE'
               WHEN b.exchcd = 2 THEN 'AMEX'
               WHEN b.exchcd = 3 THEN 'NASDAQ'
               ELSE 'Other'
           END AS exchange_name
    FROM crsp.msf AS a
    LEFT JOIN crsp.msenames AS b
        ON a.permno = b.permno
        AND b.namedt <= a.date 
        AND a.date <= b.nameendt
    WHERE a.date BETWEEN '2010-01-01' AND '2021-12-31'  -- Extending to more recent data
      AND b.shrcd IN (10, 11)
      AND b.exchcd IN (1, 2, 3)
"""
crsp_data = db.raw_sql(crsp_query)
crsp_data['date'] = pd.to_datetime(crsp_data['date'])
crsp_data['year'] = crsp_data['date'].dt.year
crsp_data['quarter'] = crsp_data['date'].dt.quarter
crsp_data['month'] = crsp_data['date'].dt.month
crsp_data['yearmonth'] = crsp_data['year'] * 100 + crsp_data['month']

# Calculate market capitalization
crsp_data['market_cap'] = crsp_data['adj_price'] * crsp_data['shares_outstanding'] / 1000  # in thousands

# Save raw CRSP data
crsp_data.to_csv(f"{base_dir}/1_raw_data/crsp_data_2010_2021.csv", index=False)
print(f"Raw CRSP data saved to {base_dir}/1_raw_data/crsp_data_2010_2021.csv")

Fetching CRSP monthly stock data...
Raw CRSP data saved to /Users/dyuan868/Desktop/gpas/4993/stat-4993-sp25/Data/1_raw_data/crsp_data_2010_2021.csv


In [8]:
# 2. Add industry classification using SIC codes
def get_industry_classification(sic_code):
    if pd.isna(sic_code):
        return 'Unknown'
    
    sic_str = str(int(sic_code)).zfill(4) if not pd.isna(sic_code) else '0000'
    
    # Define industry classifications based on SIC codes
    if sic_str.startswith(('01', '02', '07', '08', '09')):
        return 'Agriculture'
    elif sic_str.startswith(('10', '12', '13', '14')):
        return 'Mining'
    elif sic_str.startswith(('15', '16', '17')):
        return 'Construction'
    elif sic_str.startswith(('20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39')):
        return 'Manufacturing'
    elif sic_str.startswith(('40', '41', '42', '43', '44', '45', '46', '47', '48', '49')):
        return 'Transportation & Utilities'
    elif sic_str.startswith(('50', '51')):
        return 'Wholesale Trade'
    elif sic_str.startswith(('52', '53', '54', '55', '56', '57', '58', '59')):
        return 'Retail Trade'
    elif sic_str.startswith(('60', '61', '62', '63', '64', '65', '66', '67')):
        return 'Finance'
    elif sic_str.startswith(('70', '71', '72', '73', '74', '75', '76', '77', '78', '79')):
        return 'Services'
    elif sic_str.startswith(('80', '81', '82', '83', '84', '85', '86', '87', '88', '89')):
        return 'Health & Education'
    elif sic_str.startswith(('91', '92', '93', '94', '95', '96', '97', '98', '99')):
        return 'Public Administration'
    else:
        return 'Other'

# Add industry classification
crsp_data['industry'] = crsp_data['sic_code'].apply(get_industry_classification)
print(f"CRSP data loaded: {crsp_data.shape}")

# Save CRSP data with industry classification
crsp_data.to_csv(f"{base_dir}/2_processed_data/crsp_with_industry.csv", index=False)
print(f"CRSP data with industry classification saved to {base_dir}/2_processed_data/crsp_with_industry.csv")

CRSP data loaded: (538814, 19)
CRSP data with industry classification saved to /Users/dyuan868/Desktop/gpas/4993/stat-4993-sp25/Data/2_processed_data/crsp_with_industry.csv


In [14]:
# 3. Get Compustat quarterly financial data with expanded fields for advanced ratios
print("Fetching Compustat quarterly financial data...")
comp_query = """
    SELECT gvkey,
           datadate,
           fyearq AS fiscal_year,
           fqtr AS fiscal_quarter,
           rdq AS report_date,
           
           -- Income Statement Items
           revtq AS revenue,
           cogsq AS cost_of_goods_sold,
           xsgaq AS sga_expense,
           oiadpq AS operating_income,
           oibdpq AS operating_income_before_depreciation_amortization, -- EBITDA
           dpq AS depreciation_amortization,               -- Added for EBITDA calculation
           oancfy AS operating_cash_flow,
           txtq AS income_tax,
           niq AS net_income,
           piq AS pretax_income,
           ibq AS income_before_extraordinary_items,
           xintq AS interest_expense,
           dvpq AS dividends_preferred,                   -- Added for dividends
           dvpsxq AS dividends_per_share,                 -- Added for dividends yield
           
           -- Balance Sheet Items
           atq AS total_assets,
           ltq AS total_liabilities,
           ceqq AS common_equity,
           seqq AS stockholders_equity,
           cheq AS cash_and_equivalents,
           rectq AS accounts_receivable,
           invtq AS inventories, 
           ppentq AS ppe_net,
           actq AS current_assets,
           lctq AS current_liabilities,
           dlttq AS long_term_debt,
           dlcq AS debt_in_current_liabilities,
           apq AS accounts_payable,
           txpq AS taxes_payable,                        -- Added for taxes payable
           acoq AS other_current_assets,                 -- Added for other current assets
           loq AS other_liabilities,                     -- Added for other liabilities
           pstkq AS preferred_stock,
           cshoq AS common_shares_outstanding,
           
           -- Other Items
           capxy AS capital_expenditure,
           tic AS ticker_compustat
           
    FROM comp.fundq
    WHERE datadate BETWEEN '2009-01-01' AND '2021-12-31'  -- Start from 2009 to calculate YoY for 2010
      AND indfmt='INDL'
      AND datafmt='STD'
      AND popsrc='D'
      AND consol='C'
"""
comp_data = db.raw_sql(comp_query)
comp_data['datadate'] = pd.to_datetime(comp_data['datadate'])
comp_data['year'] = comp_data['datadate'].dt.year
comp_data['quarter'] = comp_data['datadate'].dt.quarter
comp_data['yearquarter'] = comp_data['year'] * 10 + comp_data['quarter']

# Save raw Compustat data
comp_data.to_csv(f"{base_dir}/1_raw_data/compustat_data_2009_2021.csv", index=False)
print(f"Raw Compustat data saved to {base_dir}/1_raw_data/compustat_data_2009_2021.csv")

Fetching Compustat quarterly financial data...
Raw Compustat data saved to /Users/dyuan868/Desktop/gpas/4993/stat-4993-sp25/Data/1_raw_data/compustat_data_2009_2021.csv


In [15]:
# 4. Link CRSP and Compustat data using CCM (CRSP/Compustat Merged Database)
print("Fetching CRSP-Compustat link table...")
ccm_query = """
    SELECT gvkey, lpermno AS permno, 
           linkdt, linkenddt
    FROM crsp.ccmxpf_linktable
    WHERE linktype IN ('LU', 'LC')
      AND linkprim IN ('P', 'C')
"""
ccm_data = db.raw_sql(ccm_query)
ccm_data['linkdt'] = pd.to_datetime(ccm_data['linkdt'])
ccm_data['linkenddt'] = pd.to_datetime(ccm_data['linkenddt'])

# Fill NaT linkenddt with future date
max_date = pd.to_datetime('2025-12-31')
ccm_data['linkenddt'] = ccm_data['linkenddt'].fillna(max_date)

# Save CCM link table
ccm_data.to_csv(f"{base_dir}/1_raw_data/ccm_link_table.csv", index=False)
print(f"CCM link table saved to {base_dir}/1_raw_data/ccm_link_table.csv")

Fetching CRSP-Compustat link table...
CCM link table saved to /Users/dyuan868/Desktop/gpas/4993/stat-4993-sp25/Data/1_raw_data/ccm_link_table.csv


In [27]:
# 5. Merging
base_dir = "/Users/dyuan868/Desktop/gpas/4993/stat-4993-sp25/Data"
crsp_file = f"{base_dir}/2_processed_data/crsp_with_industry.csv"
comp_file = f"{base_dir}/1_raw_data/compustat_data_2009_2021.csv"
ccm_file = f"{base_dir}/1_raw_data/ccm_link_table.csv"

# Read data
print("Reading CRSP, Compustat and CCM link table...")
crsp_data = pd.read_csv(crsp_file)
comp_data = pd.read_csv(comp_file)
ccm_data = pd.read_csv(ccm_file)

# Ensure date columns are in datetime format
crsp_data['date'] = pd.to_datetime(crsp_data['date'])
comp_data['datadate'] = pd.to_datetime(comp_data['datadate'])
ccm_data['linkdt'] = pd.to_datetime(ccm_data['linkdt'])
ccm_data['linkenddt'] = pd.to_datetime(ccm_data['linkenddt'])

print("Processing and merging data (optimized method)...")

# Prepare Compustat data
comp_data['year'] = comp_data['datadate'].dt.year
comp_data['month'] = comp_data['datadate'].dt.month
comp_data['quarter'] = comp_data['datadate'].dt.quarter
comp_data['yearmonth'] = comp_data['year'] * 100 + comp_data['month']

# Create a fiscal quarter identifier
comp_data['fiscal_yearquarter'] = comp_data['fiscal_year'] * 10 + comp_data['fiscal_quarter']

# Fill NaT linkenddt with future date in CCM data
max_date = pd.to_datetime('2025-12-31')
ccm_data['linkenddt'] = ccm_data['linkenddt'].fillna(max_date)

# Step 1: Merge CRSP with CCM link table
print("Step 1: Joining CRSP with CCM link table...")
crsp_ccm = pd.merge(
    crsp_data,
    ccm_data[['gvkey', 'permno', 'linkdt', 'linkenddt']],
    on='permno',
    how='inner'
)

# Filter valid links - ensure date is within link effective period
crsp_ccm = crsp_ccm[
    (crsp_ccm['date'] >= crsp_ccm['linkdt']) & 
    (crsp_ccm['date'] <= crsp_ccm['linkenddt'])
]

# Step 2: Prepare for quarter-based merging
# Create yearmonth fields for joining
crsp_ccm['year'] = crsp_ccm['date'].dt.year
crsp_ccm['month'] = crsp_ccm['date'].dt.month
crsp_ccm['yearmonth'] = crsp_ccm['year'] * 100 + crsp_ccm['month']

# Create a quarter field
crsp_ccm['quarter'] = crsp_ccm['date'].dt.quarter

# Step 3: Optimize the merging process
print("Step 3: Performing optimized merge...")

# Method 1: Using a more efficient approach with merge_asof
# Sort both dataframes by date
comp_data_sorted = comp_data.sort_values('datadate')
crsp_ccm_sorted = crsp_ccm.sort_values('date')

# Group by gvkey to process each company separately but efficiently
merged_results = []

# Get list of unique gvkeys
unique_gvkeys = crsp_ccm['gvkey'].unique()
total_gvkeys = len(unique_gvkeys)

print(f"Processing {total_gvkeys} unique companies...")

# Process in smaller batches to provide progress updates
batch_size = max(1, total_gvkeys // 10)  # Show 10 progress updates
for i, gvkey_batch in enumerate([unique_gvkeys[i:i+batch_size] for i in range(0, total_gvkeys, batch_size)]):
    batch_results = []
    
    for gvkey in gvkey_batch:
        # Get CRSP data for this gvkey
        crsp_subset = crsp_ccm_sorted[crsp_ccm_sorted['gvkey'] == gvkey]
        
        if crsp_subset.empty:
            continue
            
        # Get Compustat data for this gvkey
        comp_subset = comp_data_sorted[comp_data_sorted['gvkey'] == gvkey]
        
        if comp_subset.empty:
            continue
            
        # For each CRSP date, find the most recent Compustat date that's not after the CRSP date
        merged = pd.merge_asof(
            crsp_subset, 
            comp_subset, 
            left_on='date', 
            right_on='datadate',
            by='gvkey',
            direction='backward'
        )
        
        # Only keep rows where we found a match
        merged = merged.dropna(subset=['datadate'])
        
        # Add to batch results
        batch_results.append(merged)
    
    # Combine batch results
    if batch_results:
        batch_df = pd.concat(batch_results, ignore_index=True)
        merged_results.append(batch_df)
        
    # Show progress
    progress = min(100, int((i+1) * batch_size * 100 / total_gvkeys))
    print(f"Progress: {progress}% ({min((i+1)*batch_size, total_gvkeys)}/{total_gvkeys} companies processed)")

# Combine all results
if merged_results:
    merged_df = pd.concat(merged_results, ignore_index=True)
    print(f"Merge completed successfully, {len(merged_df)} records found.")
else:
    merged_df = pd.DataFrame()
    print("No matching records found.")

# Remove unnecessary columns
columns_to_drop = ['linkdt', 'linkenddt']
merged_df = merged_df.drop(columns=columns_to_drop, errors='ignore')

# Save merged data
output_file = f"{base_dir}/2_processed_data/crsp_compustat_merged_{datetime.now().strftime('%Y%m%d')}.csv"
merged_df.to_csv(output_file, index=False)
print(f"Merged CRSP-Compustat data saved to {output_file}")

# Data quality checks
print("\nData Quality Checks:")
print(f"Original CRSP records: {len(crsp_data)}")
print(f"Original Compustat records: {len(comp_data)}")
print(f"Merged records: {len(merged_df)}")
print(f"Unique companies (permno): {merged_df['permno'].nunique()}")
print(f"Unique companies (gvkey): {merged_df['gvkey'].nunique()}")
if not merged_df.empty:
    print(f"Data time range: {merged_df['date'].min()} to {merged_df['date'].max()}")

# Display only essential summary information to save time
print("\nSummary of merged data (count, mean, min, max of key variables):")
key_vars = ['monthly_return', 'price', 'market_cap', 'revenue', 'net_income']
key_vars = [var for var in key_vars if var in merged_df.columns]
if key_vars:
    print(merged_df[key_vars].describe().loc[['count', 'mean', 'min', 'max']])

Reading CRSP, Compustat and CCM link table...
Processing and merging data (optimized method)...
Step 1: Joining CRSP with CCM link table...
Step 3: Performing optimized merge...
Processing 7413 unique companies...
Progress: 9% (741/7413 companies processed)
Progress: 19% (1482/7413 companies processed)
Progress: 29% (2223/7413 companies processed)
Progress: 39% (2964/7413 companies processed)
Progress: 49% (3705/7413 companies processed)
Progress: 59% (4446/7413 companies processed)
Progress: 69% (5187/7413 companies processed)
Progress: 79% (5928/7413 companies processed)
Progress: 89% (6669/7413 companies processed)
Progress: 99% (7410/7413 companies processed)
Progress: 100% (7413/7413 companies processed)
Merge completed successfully, 528583 records found.
Merged CRSP-Compustat data saved to /Users/dyuan868/Desktop/gpas/4993/stat-4993-sp25/Data/2_processed_data/crsp_compustat_merged_20250314.csv

Data Quality Checks:
Original CRSP records: 538814
Original Compustat records: 599107


In [ ]:
# Disconnect from WRDS
db.close()